# Churn Modelling

This project aims to predict customer churn using a dataset and a neural network built in TensorFlow. The notebook includes data preprocessing, model building, evaluation, and predictions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


## Data Loading and Exploration

In [ ]:
# Load dataset
dataset = pd.read_csv('/content/drive/MyDrive/Deep Learning/ANN/Churn_Modelling.csv')
print(dataset.head())

# Check for missing values
print(dataset.isnull().sum())

## Data Preprocessing

In [ ]:
# Encode categorical variables
encoder = LabelEncoder()
dataset['Gender'] = encoder.fit_transform(dataset['Gender'])
dataset['Geography'] = encoder.fit_transform(dataset['Geography'])

# Feature selection
X = dataset.drop(columns=['CustomerId', 'RowNumber', 'Surname', 'Exited'])
y = dataset['Exited']

# One-hot encoding
ct = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = ct.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Model Building

In [ ]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

## Model Training

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

## Evaluation

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

## Training History Visualization

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()
    plt.title('Training History')
    plt.show()

plot_training_history(history)

## Predictions

In [ ]:
# Predict a sample
sample = scaler.transform([[0, 0, 1, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58]])
prediction = model.predict(sample)
print(f'Churn Prediction: {prediction[0][0]:.4f}')